## Face Detection

* We uses the built-in Haar cascade classifiers in OpenCV
* these classifiers have already been pre-trained to recognize faces!
* Building our own classifier is certainly outside the scope of this case study. But if we wanted to, we would need a lot of “positive” and “negative” images. Positive images would contain images with faces, whereas negative images would contain images without faces.
* Based on this dataset, we could then extract features to characterize the face (or lack of face) in an image and build our own classifier. It would be a lot work, and very time consuming.
* Anyway, these classifiers work by scanning an image from
* left to right, and top to bottom, at varying scale sizes. Scanning an image from left to right and top to bottom is called the **“sliding window”** approach.
* As the window moves from left to right and top to bottom, one pixel at a time, the classifier is asked whether or not it **“thinks”** there is a face in the current window, based on the parameters that we have supplied to the classifier
* Line 1 of the code defines FaceDetector class, which will encapsulate all the necessary logic to perform face detection.
* In **facedetector.py** the first method defines the constructor on Line 3 which takes a single parameter – the path to where his cascade classifier lives. This classifier is serialized as an XML file. Making a call to cv2.CascadeClassifier will deserialize the classifier, load it into memory, and allow us to detect faces in images.
* In **facedetector.py** the second method defines detect. This function takes one required parameter, the image that he wants to find the faces in, followed by three optional arguments. Let’s take a look at what these arguments mean:
    1. scaleFactor: How much the image size is reduced at each image scale. This value is used to create the scale pyramid. In order to detect faces at multiple scales in the image (some faces may be closer to the foreground, and thus be larger, other faces may be smaller and in the background, thus the usage of varying scales). A value of 1.05 indicates that, we are reducing the size of the image by 5% at each level in the pyramid.
    2. minNeighbors: How many neighbors each window should have for the area in the window to be considered a face. The cascade classifier will detect multiple windows around a face. This parameter controls how any rectangles (neighbors) need to be detected for the window to be labeled a face.
    3. minSize: A tuple of width and height (in pixels) indicating the minimum size of the window. Bounding boxes smaller than this size are ignored. It is a good idea to start with (30, 30) and fine tune from there.

* Detecting the actual faces in the image is handled on Line 8 by making a call to the detectMultiScale method of our classifier created in the constructor of the FaceDetector class. We supplies the scaleFactor, minNeighbors, and minSize, then the method takes care of the entire face detection process for us.

In [ ]:
from utilities.facedetector import FaceDetector
import cv2

# Define paths
image_path = 'images/obama2.png'
cascade_path = 'cascades/haarcascade_frontalface_default.xml'

# Load the image and convert it to greyscale
image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Find faces in the image
detector = FaceDetector(cascade_path)
face_boxes = detector.detect(gray, 1.2, 5)
print("{} face(s) found".format(len(face_boxes)))

# Loop over the faces and draw a rectangle around each
for (x, y, w, h) in face_boxes:
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Show the detected faces
cv2.imshow("Faces", image)
if(cv2.waitKey(0)):
 cv2.destroyAllWindows()

3 face(s) found


## Real Time Face Tracking

### Face Detector Class

```python
import cv2


class FaceDetector:
    def __init__(self, face_cascade_path):
        # Load the face detector
        self.face_cascade = cv2.CascadeClassifier(face_cascade_path)

    def detect(self, image, scale_factor=1.1, min_neighbors=5):
        # Detect faces in the image
        boxes = self.face_cascade.detectMultiScale(image, scale_factor, min_neighbors, flags=cv2.CASCADE_SCALE_IMAGE)

        # Return the bounding boxes
        return boxes

```

In [4]:
from utilities.facedetector import FaceDetector
import imutils
import cv2

# Define paths
# video_path = 'video/adrian_face.mov'
cascade_path = 'cascades/haarcascade_frontalface_default.xml'

# Construct the face detector
detector = FaceDetector(cascade_path)

# Load the video
# camera = cv2.VideoCapture(video_path)
camera = cv2.VideoCapture(0)

while True:
    # Grab the current frame
    (ok, frame) = camera.read()

    # If a frame does not exist, video is over
    if not ok:
        break

    # Resize the frame and convert it to greyscale
    frame = imutils.resize(frame, width=300)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image and clone the frame
    face_boxes = detector.detect(gray, 1.1, 5)
    clone = frame.copy()

    # Draw the face bounding boxes
    for (x, y, w, h) in face_boxes:
        cv2.rectangle(clone, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Show our detected faces
    cv2.imshow("Face", clone)

    # Ff the 'q' key is pressed, stop the loop
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Cleanup the camera and close any open windows
camera.release()
cv2.destroyAllWindows()


## Eye Tracking

* We defined our EyeTracker class on Line 3 and the constructor on Line 4.
* Our EyeTracker class takes two faceCascadePath and eyeCascadePath. The first is the path to the built-on face cascade classifier in OpenCV. The second is the path to the eye cascade classifier.
* She then loads both classifiers of disk using the cv2.CascadeClassifier function on Lines 5 and 6.
* From there, we can define the track method which is used to find the eyes in the image. This method takes only a single parameter – the image that contains the face and eyes she wants to track.
* Then we can call the detectMultiScale method (Line 9) of our faceCascade classifier. This method returns to her the bounding box locations (i.e. the x, y, width, and height) of each face in the image.
* We can then extracts the face Region of Interest (ROI) from the image on Line 16 using NumPy array slicing. The **faceROI** variable now contains the bounding box region of the face.
* This time, we makes a call to the detectMultiScale method of the eyeCascade on Line 19, giving us a list of locations in the image where eyes appear.
* We uses a much larger value of minNeighbors on Line 20 since the eye cascade tends to generate more false-positives than other classifiers.
* Then, we loops over the bounding box regions of the eyes on Line 24, and updates her list of bounding box rectangles on Line 25.Finally, the list of bounding boxes is returned to the caller on Line 28.



In [ ]:
import cv2


class EyeTracker:
    def __init__(self, face_cascade_path, eye_cascade_path):
        # load the face and eye detector
        self.face_cascade = cv2.CascadeClassifier(face_cascade_path)
        self.eye_cascade = cv2.CascadeClassifier(eye_cascade_path)

    def track(self, image):
        # Detect faces in the image and initialize the list of rectangles containing the faces and eyes
        face_boxes = self.face_cascade.detectMultiScale(image, 1.1, 5)
        boxes = []

        # Loop over the face bounding boxes
        for (f_x, f_y, f_w, f_h) in face_boxes:
            # Extract the face ROI and update the list of bounding boxes
            face_roi = image[f_y:f_y + f_h, f_x:f_x + f_w]
            boxes.append((f_x, f_y, f_x + f_w, f_y + f_h))

            # Detect eyes in the face ROI
            eye_boxes = self.eye_cascade.detectMultiScale(face_roi, 1.1, 10)

            # Loop over the eye bounding boxes
            for (e_x, e_y, e_w, e_h) in eye_boxes:
                # Update the list of bounding boxes
                boxes.append((f_x + e_x, f_y + e_y, f_x + e_x + e_w, f_y + e_y + e_h))

        # Return the bounding boxes around the faces and eyes
        return boxes


In [5]:
from utilities.eyetracker import EyeTracker
import imutils
import cv2

# Define paths
# video_path = 'video/adrian_eyes.mov'
face_cascade_path = 'cascades/haarcascade_frontalface_default.xml'
eye_cascade_path = 'cascades/haarcascade_eye.xml'

# construct the eye tracker
eye_tracker = EyeTracker(face_cascade_path, eye_cascade_path)

# Load the video
# camera = cv2.VideoCapture(video_path)
camera = cv2.VideoCapture(0)

while True:
    # Grab the current frame
    (ok, frame) = camera.read()

    # Check if at the end of the video
    if not ok:
        break

    # Resize the frame and convert it to greyscale
    # Inorder to make system more faster
    frame = imutils.resize(frame, width=300)
    # Convertingto grayscale tends to increase the accuracy of the cascade classifiers.
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces and eyes in the image
    boxes = eye_tracker.track(gray)

    # Draw the face bounding boxes
    for box in boxes:
        cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)

    # Show the tracked eyes and face
    cv2.imshow("Tracking", frame)

    # If the 'q' key is pressed, stop the loop
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Cleanup the camera and close any open windows
camera.release()
cv2.destroyAllWindows()